In [1]:
import json
import pathlib
import pydash
import rdflib
import uuid

def statement_wrapper(s,p,o,x):

    ''' Wrap statements as literals, primarily for individual encryption. '''

    meta_graph = rdflib.Graph()
    meta_graph.bind("ont", ont)
    meta_graph.bind("res", res)

    statement_id = res[str(uuid.uuid4())]

    if x == 'open':
        meta_graph.add((statement_id, rdflib.RDF.type, ont.OpenStatement))
    elif x == 'closed':
        meta_graph.add((statement_id, rdflib.RDF.type, ont.ClosedStatement))
    else:
        raise Exception('Statements must be either open or closed!')

    mini_graph = rdflib.Graph()
    mini_graph.bind("ont", ont)
    mini_graph.bind("res", res)
    mini_graph.add((s, p, o))

    payload = rdflib.Literal(str(mini_graph.serialize(format='n3')))
    meta_graph.add((statement_id, ont.hasPayload, payload))
    
    return meta_graph

with open(pathlib.Path.cwd() / 'data_in.json') as my_birth:
    my_birth = json.load(my_birth)

ont = rdflib.Namespace("https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/ontology/")
res = rdflib.Namespace("https://713527d4-9feb-4015-be1d-6d3dd68b0e89.org/resource/")

surface = rdflib.Graph()
surface.bind("ont", ont)
surface.bind("res", res)
    
surface += statement_wrapper(res[pydash.get(my_birth, 'id')], rdflib.RDF.type, ont[pydash.get(my_birth, 'event_type')], 'open')
surface += statement_wrapper(res[pydash.get(my_birth, 'id')], ont.EventDate, rdflib.Literal(pydash.get(my_birth, 'event_date')), pydash.get(my_birth, 'visible'))
surface += statement_wrapper(res[pydash.get(my_birth, 'id')], ont.EventLocation, res[pydash.get(my_birth, 'event_location.id')], pydash.get(my_birth, 'visible'))
surface += statement_wrapper(res[pydash.get(my_birth, 'event_location.id')], rdflib.RDFS.label, rdflib.Literal(pydash.get(my_birth, 'event_location.name')), pydash.get(my_birth, 'visible'))

for x in pydash.get(my_birth, 'agents'):
    activity_id = res[str(uuid.uuid4())]
    surface += statement_wrapper(res[pydash.get(my_birth, 'id')], ont.hasActivity, activity_id, pydash.get(my_birth, 'visible'))
    surface += statement_wrapper(activity_id, rdflib.RDF.type, ont[pydash.get(x, 'activity')], pydash.get(my_birth, 'visible'))
    surface += statement_wrapper(activity_id, ont.hasAgent, res[pydash.get(x, 'id')], pydash.get(my_birth, 'visible'))
    surface += statement_wrapper(res[pydash.get(x, 'id')], rdflib.RDFS.label, rdflib.Literal(pydash.get(x, 'name')), pydash.get(my_birth, 'visible'))
    surface += statement_wrapper(res[pydash.get(x, 'id')], rdflib.RDF.type, ont.Person, pydash.get(my_birth, 'visible'))
    
surface.serialize(destination=pathlib.Path.cwd() / 'data_out.json', format='json-ld')
print(len(surface), 'triples.')


38 triples.
